In [3]:
!pip -q install langchain langchain-community langchain-huggingface
!pip -q install faiss-cpu sentence-transformers pydantic
!pip -q install transformers sentencepiece


In [4]:
import os
import json
import re
import uuid
from typing import List, Dict, Any
from datetime import datetime, timezone

from pydantic import BaseModel, Field

# Create project folders (GitHub-friendly)
os.makedirs("documents", exist_ok=True)
os.makedirs("evaluation", exist_ok=True)
os.makedirs("logs", exist_ok=True)

print("✅ Folders ready:", os.listdir())


✅ Folders ready: ['.config', 'documents', 'evaluation', 'logs', 'sample_data']


In [5]:
sample_docs = {
    "payments_policy.txt": """
If a transfer fails but money is deducted, the customer must provide the transaction ID.
Support investigates within 24 hours and updates the customer.
If the transaction ID is missing, request it before escalation.
""",
    "refund_policy.txt": """
Refunds are processed within 5 business days after verification.
Duplicate charges require the order ID and payment reference.
Create a refund ticket and attach the order ID before processing.
""",
    "security_policy.txt": """
If fraud is reported or an account is suspected to be hacked:
- Escalate to the security team immediately.
- Mark the case as high priority.
- Require human confirmation before any account action.
"""
}

if len(os.listdir("documents")) == 0:
    for fname, content in sample_docs.items():
        with open(os.path.join("documents", fname), "w", encoding="utf-8") as f:
            f.write(content.strip())
    print("✅ Sample documents created:", os.listdir("documents"))
else:
    print("ℹ️ Documents already exist:", os.listdir("documents"))


ℹ️ Documents already exist: ['security_policy.txt', 'payments_policy.txt', 'refund_policy.txt']


In [6]:
from transformers import pipeline

# Lightweight model: stable, low RAM
_llm = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_new_tokens=256
)

def llm_generate(prompt: str) -> str:
    """
    Single function used everywhere.
    If you ever swap model later, change only here.
    """
    return _llm(prompt)[0]["generated_text"]

print("✅ Lightweight LLM ready")


Device set to use cpu


✅ Lightweight LLM ready


In [7]:
def extract_json(text: str) -> Dict[str, Any]:
    """
    Extract the first JSON object from text.
    If no JSON exists, raise an error (better than silent failure).
    """
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if not match:
        raise ValueError(f"No JSON found. Output was:\n{text}")
    return json.loads(match.group(0))


In [8]:
class Intake(BaseModel):
    intent: str = Field(..., description="snake_case intent like payment_failure, refund_request")
    urgency: str = Field(..., description="low|medium|high")
    missing_fields: List[str] = Field(default_factory=list, description="Required fields missing")
    confidence: float = Field(..., ge=0, le=1, description="0..1 confidence score")


In [9]:
def intake_chain(message: str) -> Intake:
    """
    Converts messy user message into structured intake info.
    Uses JSON schema to keep system reliable.
    """
    prompt = f"""
Return ONLY valid JSON with EXACT keys:
{{
  "intent": "snake_case_string",
  "urgency": "low|medium|high",
  "missing_fields": [],
  "confidence": 0.0
}}

Classify this banking support message:
- urgency=high if: money deducted, failed transfer/payment, fraud, hacked, urgent, locked account
- If transfer/payment failed and no transaction id is mentioned -> missing_fields add "transaction_id"
- If charged twice/refund and no order id is mentioned -> missing_fields add "order_id"
- confidence from 0 to 1

Message:
{message}
""".strip()

    raw = llm_generate(prompt)
    data = extract_json(raw)

    # Defensive defaults
    data.setdefault("missing_fields", [])
    data.setdefault("confidence", 0.5)

    return Intake(**data)


In [10]:
INTENT_ALIASES = {
    "transfer_failed": "payment_failure",
    "transfer_failed_urgent": "payment_failure",
    "payment_failed": "payment_failure",
    "failed_transfer": "payment_failure",
    "failed_payment": "payment_failure",

    "charged_twice": "charged_twice_refund",
    "payment_refund_charged_twice": "charged_twice_refund",

    "refund": "refund_request",
    "refund_request": "refund_request",

    "hacked_account": "account_hacked",
    "account_hacked": "account_hacked",

    "fraud": "fraud_report",
    "fraud_reported": "fraud_report",
}

def normalize_intent(raw_intent: str) -> str:
    i = (raw_intent or "").strip().lower()
    return INTENT_ALIASES.get(i, i)


In [21]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.vectorstores import FAISS

def get_embeddings():
    """
    Robust embeddings import (works across LangChain versions).
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    try:
        from langchain_huggingface import HuggingFaceEmbeddings
        return HuggingFaceEmbeddings(model_name=model_name)
    except Exception:
        from langchain_community.embeddings import HuggingFaceEmbeddings
        return HuggingFaceEmbeddings(model_name=model_name)

def build_retriever(doc_folder="documents", k=3):
    # Load documents
    docs = []
    for fname in os.listdir(doc_folder):
        if fname.lower().endswith(".txt"):
            docs.extend(TextLoader(os.path.join(doc_folder, fname), encoding="utf-8").load())

    if not docs:
        raise ValueError("❌ No .txt docs found in /documents.")

    # Chunking
    splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=80)
    chunks = splitter.split_documents(docs)

    embeddings = get_embeddings()
    vs = FAISS.from_documents(chunks, embeddings)

    r = vs.as_retriever(search_kwargs={"k": k})

    globals()["retriever"] = r
    globals()["_vectorstore"] = vs

    print(f"✅ Retriever built. Docs={len(docs)} | Chunks={len(chunks)} | k={k}")
    return r

def ensure_retriever():
    if "retriever" not in globals() or globals().get("retriever") is None:
        return build_retriever()
    return globals()["retriever"]

# Build now
ensure_retriever()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Retriever built. Docs=3 | Chunks=3 | k=3


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7ed36a3198e0>, search_kwargs={'k': 3})

In [12]:
def rag_chain(question: str) -> Dict[str, Any]:
    """
    Retrieve relevant doc chunks + answer using ONLY retrieved context.
    """
    r = ensure_retriever()
    retrieved_docs = r.get_relevant_documents(question)

    context = "\n\n".join([
        f"[Source: {d.metadata.get('source','unknown')}] {d.page_content}"
        for d in retrieved_docs
    ])

    prompt = f"""
Answer the question using ONLY the context below.
If not found, say exactly: Information not found in provided documents.

Context:
{context}

Question:
{question}

Answer:
""".strip()

    answer = llm_generate(prompt).strip()
    sources = list(dict.fromkeys([d.metadata.get("source", "unknown") for d in retrieved_docs]))
    return {"answer": answer, "sources": sources}


In [13]:
def tool_request_missing_info(missing_fields: List[str]) -> Dict[str, Any]:
    return {
        "action": "request_missing_info",
        "message_to_user": f"Please provide: {', '.join(missing_fields)}",
        "missing_fields": missing_fields
    }

def tool_create_support_ticket(intent: str, message: str) -> Dict[str, Any]:
    ticket_id = "TKT-" + str(uuid.uuid4())[:8].upper()
    return {
        "action": "create_ticket",
        "ticket_id": ticket_id,
        "intent": intent,
        "summary": message[:120]
    }

def tool_escalate_to_human(reason: str) -> Dict[str, Any]:
    return {"action": "escalate_to_human", "reason": reason}


In [14]:
MEMORY: Dict[str, Any] = {}

def get_memory(session_id: str) -> Dict[str, Any]:
    return MEMORY.get(session_id, {"history": []})

def update_memory(session_id: str, user_message: str, system_output: Dict[str, Any]) -> None:
    mem = get_memory(session_id)
    mem["history"].append({"user": user_message, "system": system_output})
    MEMORY[session_id] = mem


In [15]:
def log_decision(trace_id: str, payload: Dict[str, Any]) -> None:
    record = dict(payload)
    record["trace_id"] = trace_id
    record["time_utc"] = datetime.now(timezone.utc).isoformat()

    with open("logs/decisions.jsonl", "a", encoding="utf-8") as f:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")


In [16]:
POLICY_INTENTS = {"payment_failure", "refund_request", "charged_twice_refund", "fraud_report", "account_hacked"}

def route_decision(intake: Intake) -> str:
    if intake.confidence < 0.55:
        return "human"
    if intake.missing_fields:
        return "ask_user"
    if intake.intent in POLICY_INTENTS:
        return "rag"
    return "tool"


In [17]:
def orchestrator(message: str, session_id: str = "default") -> Dict[str, Any]:
    trace_id = str(uuid.uuid4())

    # 1) Intake
    intake = intake_chain(message)

    # 2) Normalize intent (stabilize routing)
    intake.intent = normalize_intent(intake.intent)

    # 3) Route
    route = route_decision(intake)

    # 4) Execute
    if route == "human":
        result = {
            "route": route,
            "intake": intake.model_dump(),
            "action": tool_escalate_to_human("Low confidence classification")
        }

    elif route == "ask_user":
        result = {
            "route": route,
            "intake": intake.model_dump(),
            "action": tool_request_missing_info(intake.missing_fields)
        }

    elif route == "rag":
        rag = rag_chain(message)
        result = {
            "route": route,
            "intake": intake.model_dump(),
            "rag": rag
        }

    else:
        result = {
            "route": route,
            "intake": intake.model_dump(),
            "action": tool_create_support_ticket(intake.intent, message)
        }

    # 5) Memory + logs
    update_memory(session_id, message, result)
    log_decision(trace_id, result)

    result["trace_id"] = trace_id
    return result


In [22]:
tests = [
    "My transfer failed and money was deducted urgently. My name is Fady.",
    "I was charged twice for order 9912 yesterday. Need a refund.",
    "Someone hacked my account and there are unknown transactions!",
    "Please close my account.",
    "What is the refund timeline?"
]

for t in tests:
    out = orchestrator(t, session_id="fady_demo")
    print("INPUT:", t)
    print(json.dumps(out, indent=2))
    print("-"*90)

print("✅ Memory items:", len(get_memory("fady_demo")["history"]))


ValueError: No JSON found. Output was:
urgency=high

In [19]:
test_cases = [
    {"message": "Transfer failed and money deducted.", "expected": ["ask_user", "rag"]},
    {"message": "Charged twice for order 9912 need refund.", "expected": ["rag", "ask_user"]},
    {"message": "My account is hacked!", "expected": ["rag", "human"]},
    {"message": "Close my account please.", "expected": ["tool"]},
    {"message": "What is the refund timeline?", "expected": ["rag"]},
]

with open("evaluation/test_cases.json", "w", encoding="utf-8") as f:
    json.dump(test_cases, f, ensure_ascii=False, indent=2)

print("✅ Saved evaluation/test_cases.json")


✅ Saved evaluation/test_cases.json


In [20]:
with open("evaluation/test_cases.json", "r", encoding="utf-8") as f:
    cases = json.load(f)

results = []
passed = 0

for c in cases:
    output = orchestrator(c["message"], session_id="eval")
    route = output.get("route", "unknown")

    ok = route in c["expected"]
    passed += int(ok)

    results.append({
        "message": c["message"],
        "expected": c["expected"],
        "predicted_route": route,
        "ok": ok
    })

report = {
    "tests": len(results),
    "passed": passed,
    "pass_rate": passed / len(results)
}

with open("evaluation/results.json", "w", encoding="utf-8") as f:
    json.dump({"report": report, "results": results}, f, ensure_ascii=False, indent=2)

print("✅ Evaluation report:", report)
results


ValueError: No JSON found. Output was:
urgency=high